In [ ]:
key =''

In [ ]:
!pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import openai
from openai import OpenAI

In [ ]:
openai.api_key=key
client=OpenAI(api_key=key)

In [ ]:
import pandas as pd

df=pd.read_csv('/content/output_questions (2).tsv', sep='\t')
df.drop(columns=['Correct Answer','Explanation'], inplace=True)
df.head()


,Question,Option A,Option B,Option C,Option D
0,Which MITRE ATT&CK technique involves an adver...,Spearphishing Attachment (T1193),Social Engineering (T1140),Remote Access Tools (T1219),Man-in-the-Middle Attack (T1557)
1,Which MITRE ATT&CK technique involves an adver...,(A) Exploit Public-Facing Application (T1190),(B) Third-party Software (T1072),(C) Exploitation for Client Execution (T1203),(D) Exploitation for Privilege Escalation (T1068)
2,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Data Staged (T1074),Command and Control (T1009),Remote File Copy (T1105)
3,"In the context of MITRE ATT&CK for Enterprise,...",T1548.002 - Abuse Elevation Control Mechanism,T1059.001 - Command-Line Interface,T1566.001 - Phishing,T1078.002 - Valid Accounts
4,Which MITRE ATT&CK technique involves an adver...,Process Injection (T1055) - Process Injection,Man-in-the-Middle (T1557) - Man-in-the-Middle,Control Device Identification (T818) - Control...,Manipulate Device Communication (T848) - Manip...


In [ ]:
import os
import openai
from openai import OpenAI

def get_gpt35_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in cybersecurity and CWE mappings."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )

    # Extract and return the generated GT
    return response.choices[0].message['content'].strip()


def answer_mcq(question, options):
   prompt = f"""
    Analyze the following CVE description and map it to the appropriate CWE. Provide a brief justification for your choice. Ensure the last line of your response contains only the CWE ID.
    CVE Description: {cve_description}
    """
    for i, option in enumerate(options):
        prompt += f"{chr(65 + i)}. {option}\n"
    prompt += "Answer:"

    response = get_gpt35_response(prompt)
    # Extract the answer letter (e.g., "A" from "A. To redirect legitimate traffic to a malicious server")
    answer_letter = response.split('.')[0].strip().upper() if '.' in response else response.strip().upper()
    return answer_letter

def process_csv(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df_sample = df.sample(n=250, random_state=1)
    responses = []

    for _, row in df_sample.iterrows():
        question = row['Question']
        options = [row['Option A'], row['Option B'], row['Option C'], row['Option D']]
        response = answer_mcq(question, options)
        responses.append(response)

    df_sample['GPT-3.5'] = responses
    return df_sample



In [ ]:
# Example usage
file_path = '/content/output_questions (2).tsv'  # Replace with the path to your CSV file
result_df = process_csv(file_path)
print(result_df[['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'GPT-3.5']])

                                               Question  \
995   Which MITRE ATT&CK technique involves an adver...   
1104  Which MITRE ATT&CK technique involves an adver...   
692   Which MITRE ATT&CK technique involves an adver...   
1452  Which MITRE ATT&CK technique involves an adver...   
547   Which MITRE ATT&CK technique involves an adver...   
...                                                 ...   
65    Which MITRE ATT&CK technique, often used in pr...   
1596  Which MITRE ATT&CK technique is associated wit...   
1165  In the context of MITRE ATT&CK for Enterprise,...   
1147  Which MITRE ATT&CK technique is described as u...   
1593  Which MITRE ATT&CK technique involves an attac...   

                                               Option A  \
995                                      SPEAR PHISHING   
1104                 Discovery: Active Scanning (T1046)   
692                                      Reconnaissance   
1452                         Mitigating Control Evasion

In [ ]:
output_file_path = '/content/GPT3.csv'  # Replace with the desired output path
result_df.to_csv(output_file_path, index=False)

In [ ]:
gpt=pd.read_csv("/content/GPT3.csv")
gpt.head()

,Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation,GPT-3.5
0,Which MITRE ATT&CK technique involves an adver...,SPEAR PHISHING,VALID ACCOUNTS,APPLICATION LAYER PROTOCOL,MAN-IN-THE-MIDDLE,C,APPLICATION LAYER PROTOCOL (T1071) involves ad...,C
1,Which MITRE ATT&CK technique involves an adver...,Discovery: Active Scanning (T1046),Delivery: Spearphishing Attachment (T1193),Execution: Command and Scripting Interpreter (...,Defense Evasion: Masquerading (T1036),A,Adversaries may use Discovery: Active Scanning...,A
2,Which MITRE ATT&CK technique involves an adver...,Reconnaissance,Exploitation Collection,Discovery,B,Adversaries often perform Discovery techniques...,NaN,C
3,Which MITRE ATT&CK technique involves an adver...,Mitigating Control Evasion,VMware vSphere Session Hijacking,Subverting Trust Controls,Hypervisor Software Backdoor,C,Adversaries can exploit vulnerabilities in vir...,D
4,Which MITRE ATT&CK technique involves an adver...,Reconnaissance - Active Scanning,Valid Accounts,Account Discovery,Network Service Scanning,Account Discovery,Adversaries may use valid accounts to interact...,C


In [ ]:
def calculate_accuracy(df):
    # Compare model responses to ground truth
    correct = (df['GPT-3.5'].str.strip().str.upper() == df['Correct Answer'].str.strip().str.upper()).sum()
    total = len(df)
    accuracy = correct / total * 100
    return accuracy

In [ ]:
accuracy = calculate_accuracy(result_df)
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 62.00%


In [ ]:
def compute_mcq_accuracy(df):
    correct = 0
    total = 0

    for idx, row in df.iterrows():
        pred = row['GPT-3.5']
        gt = row['Correct Answer']

        # Check if both pred and gt are not NaN
        if pd.notna(pred) and pd.notna(gt):
            pred = pred.strip().upper()
            gt = gt.strip().upper()

            if pred in ['A', 'B', 'C', 'D']:
                total += 1
                if pred == gt:
                    correct += 1
            else:
                print(f'Invalid response at row {idx+1}: {pred}')
        else:
            print(f'Missing value at row {idx+1}')

    accuracy = (correct / total) * 100 if total > 0 else 0
    return accuracy

accuracy = compute_mcq_accuracy(gpt)
print(f'Accuracy: {accuracy:.2f}%')


Missing value at row 49
Missing value at row 100
Missing value at row 131
Missing value at row 132
Missing value at row 148
Missing value at row 180
Accuracy: 63.52%


In [ ]:
compute_mcq_accuracy(gpt, 'GPT-3.5')

KeyError: 'GPT-3.5'